## 评估模型

In [ ]:

data=[
    {
        "mol": "NS(=O)(=O)c1cc2ccccc2o1",
        "protein": "HHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK",
        "text": 8.64,
        "pocket": "Q90,H92,H94,E104,H117,V119,F128,V140,S194,L195,T196,T197,P198,P199,L200,W206",
        "pdb_code": "3s71",
        "resolution": "1.25",
        "release_year": 2011,
        "binding_info": "Kd=2.3nM",
        "ligand_name": "EVD"
    },
    {
        "mol": "Cc1nnc(N2CCN(c3ncnc4sc(CC(F)(F)F)cc34)CC2)s1",
        "protein": "GLKAAQKTLFPLRSIDDVVRLFAAELGREEPDLVLLSLVLGFVEHFLAVNRVGLTYFPVADLSIIAALYARFTAQIRGAVDLSLYPREGGVSSRELVKKVSDVIWNSLSRSYFKDRAHIQSLFSFITGTKLDSSGVAFAVVGACQALGLRDVHLALSEDHAWVVFGPNGEQTAEVTWHGKGNEDRRGQTVNAGVAERSWLYLKGSYMRCDRKMEVAFMVCAINPSIDLHTDSLELLQLQQKLLWLLYDLGHLERYPMALGNLADLEELEPTPGRPDPLTLYHKGIASAKTYYRDEHIYPYMYLAGYHCRNRNVREALQAWADTATVIQDYNYCREDEEIYKEFFEVANDVIPNLLKEAASLLEAGSQGSALQDPECFAHLLRFYDGICKWEEGSPTPVLHVGWATFLVQSLGRFEGQVRQKVRIVSPVLTFQSEKMKGMKELLVATKINSSAIKLQLTAQSQVQMKK",
        "text": 6.11,
        "pocket": "S133,S134,L156,S157,E158,D159,H160,A161,F217,C220,Y255,M257,A258,N261,D264,Y298,M301,Y302",
        "pdb_code": "5dd9",
        "resolution": "1.62",
        "release_year": 2015,
        "binding_info": "IC50=779nM",
        "ligand_name": "59K"
    },
    {
        "mol": "O=C(N[C@H]1CC[N@H+](Cc2ccccc2)CC1)c1ccc(I)cc1",
        "protein": "QWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLARQYAGLDHELAFSRLIVELRRLHPGHVLPDEELQWVFVNAGGWMGAMCLLHASLSEYVLLFGTALGSRGHSGRYWAEISDTIISGTFHQWREGTTKSEVFYPGETVVHGPGEATAVEWGPNTWMVEYGRGVIPSTLAFALADTVFSTQDFLTLFYTLRSYARGLRLELTTYL",
        "text": 8.59,
        "pocket": "V83,W88,M92,L94,A97,Y102,L104,F106,S116,Y119,I123,D125,F132,H153,V161,W163,E171,I177,T180,L181,A184,T201,Y205",
        "pdb_code": "5hk2",
        "resolution": "3.20",
        "release_year": 2016,
        "binding_info": "Ki=2.6nM",
        "ligand_name": "61V"
    }]


In [ ]:
import sys
import os
os.chdir("/data01/luog/pmllm")

import yaml
import torch
from omegaconf import OmegaConf
from taming.models.model_pmllm_v4 import PmllmModel


DEVICE = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")


def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_model(config, ckpt_path=None):
    model = PmllmModel(**config.model.params)
    if ckpt_path is not None:
        model.init_from_ckpt(ckpt_path)
    return model.eval()


config_path="/data01/luog/pmllm/config/pmllm_pocket/DTA/Pocket_v4_bert_large_esm2_molformer.yaml"
ckpt_path="/data01/luog/pmllm/logs/2024-12-22T09-59-43_Pocket_v4_bert_large_esm2_molformer/checkpoints/last.ckpt"

config=load_config(config_path)
model=load_model(config, ckpt_path)


In [ ]:

p=[i["protein"] for i in data]
m=[i["mol"] for i in data]
t=[i["text"] for i in data]
pocket=[]
for x in data:
    pocket.append([i[1:] for i in x["pocket"].split(",")])

model.training_mask=2
outputs_pm, y_m, text_pre, t_loss, pocket = model(p,m,t,pocket)
print(model.training_mask)
p_probs, m_probs  = model.loss.pm_decoder(outputs_pm) 

m_labels = y_m.reshape(-1) # 分子SMILES label
m_pre = m_probs.argmax(dim=-1).reshape(-1)

p_labels = pocket.reshape(-1) # 口袋残基token
p_pre = p_probs.argmax(dim=-1).reshape(-1)

print("m_labels",m_labels)
print("m_pre",m_pre)
print("p_labels",p_labels)
print("p_pre",p_pre)

In [ ]:
import json

input_file = "/root/private_data/luog/AbAgker/data/origin_json/skempi_v2_ed.json"
Kd_all=[]
pdb=[]

with open(input_file,"r") as f:
    nums_i=json.load(f)
    for i in nums_i:
        if len(i["seqs"])==2 and i["kd"]!="": 
            Kd_all.append(i)
            if i["pdb"] not in pdb:
                pdb.append(i["pdb"])

print(len(Kd_all))
print(len(Kd_all))

In [3]:
from transformers import AutoModel, AutoConfig

# 指定模型的路径
model_path = '/root/private_data/luog/pretrained_LLM/palm/A2binder_affinity/lightmodel'

# 从 config.json 文件中加载模型配置
config = AutoConfig.from_pretrained(model_path)

# 从 pytorch_model.bin 文件中加载预训练模型
model = AutoModel.from_pretrained(model_path, config=config)

# 打印模型结构
print(model)


RoFormerModel(
  (embeddings): RoFormerEmbeddings(
    (word_embeddings): Embedding(25, 768, padding_idx=20)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RoFormerEncoder(
    (embed_positions): RoFormerSinusoidalPositionalEmbedding(512, 64)
    (layer): ModuleList(
      (0): RoFormerLayer(
        (attention): RoFormerAttention(
          (self): RoFormerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RoFormerSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (d